In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 79952, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 79952 (delta 46), reused 43 (delta 18), pack-reused 79824
Receiving objects: 100% (79952/79952), 594.17 MiB | 48.98 MiB/s, done.
Resolving deltas: 100% (56889/56889), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!pip uninstall imgaug
!pip install imgaug==0.4.0

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import numpy as np
import os
import shutil
import random
from lxml import etree
import re
import xml.etree.ElementTree as ET

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
import tensorflow as tf

In [ ]:
# from tensorflow_datasets.core.utils.lazy_imports_utils import tensorflow as tf
# import tensorflow_datasets.public_api as tfds

In [ ]:
from object_detection.utils import dataset_util

In [ ]:
!mkdir dataset

In [ ]:
%cd dataset

/content/dataset


In [ ]:
!tar -xvf /content/drive/MyDrive/dataset/oxford_pet/annotations.tar.gz

In [ ]:
!tar -xvf /content/drive/MyDrive/dataset/oxford_pet/images.tar.gz

In [ ]:
data_dir = "/content/dataset"

In [ ]:
%cd /content

/content


In [ ]:
image_dir = os.path.join(data_dir, 'images')
annotations_dir = os.path.join(data_dir, 'annotations')
examples_path = os.path.join(annotations_dir, 'trainval.txt')
examples_list = dataset_util.read_examples_list(examples_path)

In [ ]:
print(image_dir)
print(annotations_dir)
print(examples_path)
print(examples_list)
print("examples_list size=", len(examples_list))

In [ ]:
# Test images are not included in the downloaded data set, so we shall perform
# our own split.
random.seed(42)
random.shuffle(examples_list)
num_examples = len(examples_list)
num_train = int(0.7 * num_examples)
train_examples = examples_list[:num_train]
val_examples = examples_list[num_train:]
print('%d training and %d validation examples.',
              len(train_examples), len(val_examples))

%d training and %d validation examples. 2576 1104


In [ ]:
use_custom_dataset=True

In [ ]:
dataset_is_split=True

In [ ]:
if use_custom_dataset:
  
  # Your labels map as a dictionary (zero is reserved):
  label_map = {1: 'Abyssinian', 2: 'american_bulldog', 3: 'american_pit_bull_terrier', 4: 'basset_hound', 5: 'beagle',
               6: 'Bengal', 7: 'Birman', 8: 'Bombay', 9: 'boxer', 10: 'British_Shorthair',
               11: 'chihuahua', 12: 'Egyptian_Mau', 13: 'english_cocker_spaniel', 14: 'english_setter', 15: 'german_shorthaired', 
               16: 'great_pyrenees', 17: 'havanese', 18: 'japanese_chin', 19: 'keeshond', 20: 'leonberger',
               21: 'Maine_Coon', 22: 'miniature_pinscher', 23: 'newfoundland', 24: 'Persian', 25: 'pomeranian',
               26: 'pug', 27: 'Ragdoll', 28: 'Russian_Blue', 29: 'saint_bernard', 30: 'samoyed',
               31: 'scottish_terrier', 32: 'shiba_inu', 33: 'Siamese', 34: 'Sphynx', 35: 'staffordshire_bull_terrier',
               36: 'wheaten_terrier', 37: 'yorkshire_terrier' } 

  if dataset_is_split:
    # If your dataset is already split, specify each path:
    train_images_dir = '/content/dataset/train/images'
    train_annotations_dir = '/content/dataset/train/annotations'
    val_images_dir = '/content/dataset/validation/images'
    val_annotations_dir = '/content/dataset/validation/annotations'
    test_images_dir = '/content/dataset/test/images'
    test_annotations_dir = '/content/dataset/test/annotations'
  else:
    # If it's NOT split yet, specify the path to all images and annotations
    images_in = '/content/dataset/images'
    annotations_in = '/content/dataset/annotations'
  

In [ ]:
!mkdir "/content/dataset/train"

In [ ]:
!mkdir "/content/dataset/train/images"

In [ ]:
!mkdir "/content/dataset/train/annotations"

In [ ]:
!mkdir "/content/dataset/validation"

In [ ]:
!mkdir "/content/dataset/validation/images"

In [ ]:
!mkdir '/content/dataset/validation/annotations'

In [ ]:
def get_class_name_from_filename(file_name):
  """Gets the class name from a file.

  Args:
    file_name: The file name to get the class name from.
               ie. "american_pit_bull_terrier_105.jpg"

  Returns:
    A string of the class name.
  """
  match = re.match(r'([A-Za-z_]+)(_[0-9]+\.jpg)', file_name, re.I)
  return match.groups()[0]

In [ ]:
def adjust_to_annotation(data,target_path):
  """Notice that this function normalizes the bounding box coordinates provided
  by the raw data.

  Args:
    data: dict holding PASCAL XML fields for a single image (obtained by
      running dataset_util.recursive_parse_xml_to_dict)
    
    label_map_dict: A map from string label names to integers ids.
    image_subdirectory: String specifying subdirectory within the
      Pascal dataset directory holding the actual image data.
    ignore_difficult_instances: Whether to skip difficult instances in the
      dataset  (default: False).
    faces_only: If True, generates bounding boxes for pet faces.  Otherwise
      generates bounding boxes (as well as segmentations for full pet bodies).
    

  Returns:
    example: The converted tf.Example.

  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """
    
  class_name = get_class_name_from_filename(data['filename']) 
  targetXML = open(target_path, 'rt', encoding='UTF8')
  tree = ET.parse(targetXML)

  root = tree.getroot() #

  for obj in root.findall('object'): 
    
    name_tag = obj.find('name')
    
    name_tag.text = class_name 
  
  tree.write(target_path)  

  
  

In [ ]:
# xml_file = os.path.join(ANNO_DIR, '2007_000032.xml')

# #XML 파일을 Parsing 하여 Element 생성
# tree = ET.parse(xml_file) #트리를 만든다
# root = tree.getroot() #루트를 가져온다

# #image 관련 정보는 root의 자식으로 존재
# image_name = root.find('filename').text #루트에서 파일 이름 가져온다
# full_image_name = os.path.join(IMAGE_DIR, image_name)
# image_size = root.find('size')
# image_width = int(image_size.find('width').text) #size에서 다시 find를 하면 size의 width를 가져온다
# image_height = int(image_size.find('height').text)

# #파일내에 있는 모든 object Element를 찾음
# objects_list = []
# for obj in root.findall('object'): #모든 object
#     #object element의 자식 element에서 bndbox를 찾음
#     name_tag = obj.find('name')
    
    
    
#     class_name=obj.find('name').text
#     object_dict={'class_name': class_name, 'bndbox_pos':bndbox_pos} #딕셔너리
#     objects_list.append(object_dict) #리스트에 넣는다

In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
for idx, example in enumerate(train_examples):
      xml_path = os.path.join(annotations_dir, 'xmls', example + '.xml')
      # print(xml_path)
      if not os.path.exists(xml_path):
        print("no xml...:", xml_path)
        continue

      with tf.gfile.GFile(xml_path, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

      img_path = os.path.join(image_dir, data['filename'])
      # print(img_path)

      trainimgsource = img_path
      trainimgdestination = os.path.join(train_images_dir, data['filename'])
      shutil.copyfile(trainimgsource, trainimgdestination)
      # print(destination)

      trainxmlsource = xml_path
      trainxmldestination = os.path.join(train_annotations_dir, example + '.xml')
      shutil.copyfile(trainxmlsource, trainxmldestination)

      with tf.gfile.GFile(trainxmldestination, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      
      adjust_to_annotation(data, trainxmldestination)
      

In [ ]:
for idx, example in enumerate(val_examples):
      xml_path = os.path.join(annotations_dir, 'xmls', example + '.xml')
      # print(xml_path)
      if not os.path.exists(xml_path):
        print("no xml...:", xml_path)
        continue

      with tf.gfile.GFile(xml_path, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

      img_path = os.path.join(image_dir, data['filename'])
      # print(img_path)

      valimgsource = img_path
      valimgdestination = os.path.join(val_images_dir, data['filename'])
      shutil.copyfile(valimgsource, valimgdestination)
      # print(destination)

      valxmlsource = xml_path
      valxmldestination = os.path.join(val_annotations_dir, example + '.xml')
      shutil.copyfile(valxmlsource, valxmldestination)

      with tf.gfile.GFile(valxmldestination, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      adjust_to_annotation(data, valxmldestination)
      

no xml...: /content/dataset/annotations/xmls/Bengal_111.xml


In [ ]:
# We need to instantiate a separate DataLoader for each split dataset
if use_custom_dataset:
  if dataset_is_split:
    train_data = object_detector.DataLoader.from_pascal_voc(
        train_images_dir, train_annotations_dir, label_map=label_map)
    validation_data = object_detector.DataLoader.from_pascal_voc(
        val_images_dir, val_annotations_dir, label_map=label_map)
    # test_data = object_detector.DataLoader.from_pascal_voc(
    #     test_images_dir, test_annotations_dir, label_map=label_map)
# 내부에서 tf.record를 만들어주는 함수

  else:
    train_dir, val_dir, test_dir = split_dataset(images_in, annotations_in,
                                                 val_split=0.2, test_split=0.2,
                                                 out_path='split-dataset')
    train_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(train_dir, 'images'),
        os.path.join(train_dir, 'annotations'), label_map=label_map)
    validation_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(val_dir, 'images'),
        os.path.join(val_dir, 'annotations'), label_map=label_map)
    test_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(test_dir, 'images'),
        os.path.join(test_dir, 'annotations'), label_map=label_map)
    
  print(f'train count: {len(train_data)}')
  print(f'validation count: {len(validation_data)}')
  # print(f'test count: {len(test_data)}')

train count: 2568
validation count: 1103


In [ ]:
import tensorflow as tf

In [ ]:
spec = object_detector.EfficientDetLite1Spec()

In [ ]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=125, 
                               batch_size=16, 
                               train_whole_model=True)

Epoch 1/125
160/160 [==============================] - 129s 532ms/step - det_loss: 1.7759 - cls_loss: 1.4619 - box_loss: 0.0063 - reg_l2_loss: 0.0768 - loss: 1.8527 - learning_rate: 0.0140 - gradient_norm: 1.6615 - val_det_loss: 1.3558 - val_cls_loss: 1.1342 - val_box_loss: 0.0044 - val_reg_l2_loss: 0.0770 - val_loss: 1.4328
Epoch 2/125
160/160 [==============================] - 83s 518ms/step - det_loss: 1.0239 - cls_loss: 0.8779 - box_loss: 0.0029 - reg_l2_loss: 0.0772 - loss: 1.1012 - learning_rate: 0.0200 - gradient_norm: 2.2835 - val_det_loss: 1.0131 - val_cls_loss: 0.8266 - val_box_loss: 0.0037 - val_reg_l2_loss: 0.0775 - val_loss: 1.0907
Epoch 3/125
160/160 [==============================] - 83s 517ms/step - det_loss: 0.8418 - cls_loss: 0.7189 - box_loss: 0.0025 - reg_l2_loss: 0.0777 - loss: 0.9196 - learning_rate: 0.0200 - gradient_norm: 2.1690 - val_det_loss: 0.8463 - val_cls_loss: 0.6714 - val_box_loss: 0.0035 - val_reg_l2_loss: 0.0779 - val_loss: 0.9243
Epoch 4/125
160/160 [

In [ ]:
model.evaluate(validation_data)

18/18 [==============================] - 77s 2s/step



{'AP': 0.76829636,
 'AP50': 0.883775,
 'AP75': 0.85657305,
 'APs': -1.0,
 'APm': 0.7086178,
 'APl': 0.78909653,
 'ARmax1': 0.8303958,
 'ARmax10': 0.85381204,
 'ARmax100': 0.85440767,
 'ARs': -1.0,
 'ARm': 0.7793835,
 'ARl': 0.8683044,
 'AP_/Abyssinian': 0.73773265,
 'AP_/american_bulldog': 0.69163704,
 'AP_/american_pit_bull_terrier': 0.5604282,
 'AP_/basset_hound': 0.7807417,
 'AP_/beagle': 0.62985563,
 'AP_/Bengal': 0.70393306,
 'AP_/Birman': 0.764478,
 'AP_/Bombay': 0.90814966,
 'AP_/boxer': 0.76699626,
 'AP_/British_Shorthair': 0.90888387,
 'AP_/chihuahua': 0.6612376,
 'AP_/Egyptian_Mau': 0.6743122,
 'AP_/english_cocker_spaniel': 0.70764196,
 'AP_/english_setter': 0.8463919,
 'AP_/german_shorthaired': 0.8143677,
 'AP_/great_pyrenees': 0.7892545,
 'AP_/havanese': 0.66640836,
 'AP_/japanese_chin': 0.69545984,
 'AP_/keeshond': 0.8302783,
 'AP_/leonberger': 0.8198744,
 'AP_/Maine_Coon': 0.80412185,
 'AP_/miniature_pinscher': 0.6101174,
 'AP_/newfoundland': 0.758459,
 'AP_/Persian': 0.8

In [ ]:
TFLITE_FILENAME = 'efficientdet-lite-oxfordIII.tflite'
LABELS_FILENAME = 'oxfordIII-labels.txt'

In [ ]:
model.export(export_dir='.', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

In [ ]:
model.evaluate_tflite(TFLITE_FILENAME, validation_data)

1103/1103 [==============================] - 6032s 5s/step



{'AP': 0.73396754,
 'AP50': 0.84790933,
 'AP75': 0.8229456,
 'APs': -1.0,
 'APm': 0.6869158,
 'APl': 0.75262946,
 'ARmax1': 0.7773169,
 'ARmax10': 0.77796465,
 'ARmax100': 0.77796465,
 'ARs': -1.0,
 'ARm': 0.7083427,
 'ARl': 0.79372704,
 'AP_/Abyssinian': 0.69222605,
 'AP_/american_bulldog': 0.6329601,
 'AP_/american_pit_bull_terrier': 0.5553651,
 'AP_/basset_hound': 0.7682777,
 'AP_/beagle': 0.59801847,
 'AP_/Bengal': 0.6319874,
 'AP_/Birman': 0.6886465,
 'AP_/Bombay': 0.89593303,
 'AP_/boxer': 0.72366774,
 'AP_/British_Shorthair': 0.8939814,
 'AP_/chihuahua': 0.5546516,
 'AP_/Egyptian_Mau': 0.6446493,
 'AP_/english_cocker_spaniel': 0.684677,
 'AP_/english_setter': 0.7762489,
 'AP_/german_shorthaired': 0.7681648,
 'AP_/great_pyrenees': 0.6804982,
 'AP_/havanese': 0.6567277,
 'AP_/japanese_chin': 0.69678605,
 'AP_/keeshond': 0.82249,
 'AP_/leonberger': 0.80431336,
 'AP_/Maine_Coon': 0.7574393,
 'AP_/miniature_pinscher': 0.5424702,
 'AP_/newfoundland': 0.6872431,
 'AP_/Persian': 0.81209

In [ ]:
val_images_dir = '/content/dataset/validation/images'
val_annotations_dir = '/content/dataset/validation/annotations'
test_images_dir = '/content/dataset/validation/images'
test_annotations_dir = '/content/dataset/validation/annotations'

In [ ]:
import random

# If you're using a custom dataset, we take a random image from the test set:
if use_custom_dataset:
  images_path = test_images_dir if dataset_is_split else os.path.join(test_dir, "images")
  filenames = os.listdir(os.path.join(images_path))
  random_index = random.randint(0,len(filenames)-1)
  INPUT_IMAGE = os.path.join(images_path, filenames[random_index])
# else:
#   # Download a test salad image
#   INPUT_IMAGE = 'salad-test.jpg'
#   DOWNLOAD_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg"
#   !wget -q -O $INPUT_IMAGE $DOWNLOAD_URL

In [ ]:
INPUT_IMAGE="/content/dataset/validation/images/Abyssinian_183.jpg"

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://google-coral.github.io/py-repo/
     |████████████████████████████████| 380 kB 3.5 MB/s 
     |████████████████████████████████| 1.5 MB 2.0 MB/s 


In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILENAME)
interpreter = tflite.Interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open(INPUT_IMAGE)
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.4, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image